The main goal of this notebook is to explore SV simulated outputs
from the SV data challenge `svdc2019c`

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from astropy.table import Table
from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask, mws_mask
from collections import Counter

%matplotlib inline

First some environment definitions

In [2]:
sv_main_dir = '/global/cscratch1/sd/mjwilson/svdc2019c/' 
fiberassign_sub_dir = 'fiberassign/SV_BGS/'
fib_dir = os.path.join(sv_main_dir, fiberassign_sub_dir)

Read the input catalog data

In [3]:
mtl_bgs = Table.read(os.path.join(sv_main_dir, 'targets', 'mtl-SV_BGS.fits'))

Make a list of all available tiles

In [4]:
tile_names = glob.glob(fib_dir+"/tile*.fits")
print(fib_dir)
n_tiles = len(tile_names)

/global/cscratch1/sd/mjwilson/svdc2019c/fiberassign/SV_BGS/


In [5]:
print('Total number of tiles {}'.format(n_tiles))

Total number of tiles 28


Explore the first tile.

Find the kind of samples present in the assigned fibers

In [12]:
tile_id = 0 
tilefile = tile_names[0]
assignments = Table.read(tilefile, 'FIBERASSIGN')
potential = Table.read(tilefile, 'POTENTIAL_ASSIGNMENTS')

n_assignments = len(assignments)

In [13]:
desi_classes_in_assignment = []
bgs_classes_in_assignment = []
mws_classes_in_assignment = []

for i in range(n_assignments):
    desi_classes_in_assignment.extend(desi_mask.names(assignments['SV1_DESI_TARGET'][i]))
    bgs_classes_in_assignment.extend(bgs_mask.names(assignments['SV1_BGS_TARGET'][i]))
    mws_classes_in_assignment.extend(mws_mask.names(assignments['SV1_MWS_TARGET'][i]))

In [14]:
print(Counter(desi_classes_in_assignment))
print(Counter(bgs_classes_in_assignment))
print(Counter(mws_classes_in_assignment))

Counter({'BGS_ANY': 3990, 'SKY': 801, 'MWS_ANY': 175, 'STD_BRIGHT': 160, 'STD_FAINT': 137, 'STD_WD': 40})
Counter({'BGS_BRIGHT': 3680, 'BGS_FAINT': 244, 'BGS_FAINT_EXT': 66, 'BGS_FIBMAG': 11})
Counter({'MWS_MAIN_BLUE': 148, 'MWS_WD': 40, 'MWS_NEARBY': 5})


Find the samples that were available to the fibers

In [15]:
desi_classes_in_potential = []
bgs_classes_in_potential = []
mws_classes_in_potential = []

ii = np.in1d(mtl_bgs['TARGETID'], potential['TARGETID'])
mtl_bgs_available = mtl_bgs[ii]
print(len(mtl_bgs_available))
for t in mtl_bgs_available[['SV1_DESI_TARGET', 'SV1_BGS_TARGET','SV1_MWS_TARGET']]:
    desi_classes_in_potential.extend(desi_mask.names(t['SV1_DESI_TARGET']))
    bgs_classes_in_potential.extend(bgs_mask.names(t['SV1_BGS_TARGET']))
    mws_classes_in_potential.extend(mws_mask.names(t['SV1_MWS_TARGET']))

19240


In [16]:
print(Counter(desi_classes_in_potential))
print(Counter(bgs_classes_in_potential))
print(Counter(mws_classes_in_potential))

Counter({'BGS_ANY': 17798, 'STD_BRIGHT': 1401, 'STD_FAINT': 943, 'MWS_ANY': 923, 'STD_WD': 41})
Counter({'BGS_FAINT': 8093, 'BGS_BRIGHT': 7835, 'BGS_FAINT_EXT': 1870, 'BGS_FIBMAG': 263})
Counter({'MWS_MAIN_BLUE': 889, 'MWS_WD': 41, 'MWS_NEARBY': 5})
